<a href="https://colab.research.google.com/github/mihaidobri/Deep_Learning_Sentiment_Analysis/blob/master/Simple_MLP_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CNN Demo

How to create a CNN with PyTorch to classify a finite number of classes

NOTE: If you run into import errors, that means you're probably missing some dependencies. You can create the same conda environment I use by using the file req.txt included and running this terminal command:

`conda create -n new_environment_name --file req.txt`

### Prepare the Data
The first step to train a neural network is to prepare the data. Here we are going to be using the CIFAR (Canadian Institute for Advanced Research) 10 dataset. The dataset includes 60,000 images of low resolution images each belonging to one of 10 distinct classes.

<img src='assets/cifar10_plot.png' style="width: 75%; height: 75%">

### Splitting the data
- 50,000 for training
- 10,000 for testing

<br>
This means the neural network will learn from the 50,000 images and then we will test its performance against the 10,000 images it has never seen before.

In [2]:
import numpy as np
import torch
from torchvision import datasets, transforms

transform = transforms.Compose([transforms.RandomVerticalFlip(),
                                transforms.RandomRotation(20),
                                transforms.RandomHorizontalFlip(),
                                transforms.ColorJitter(),
                                transforms.ToTensor(),
                                transforms.Normalize((0.5, 0.5, 0.5),
                                                     (0.5, 0.5, 0.5))])

test_transform = transforms.Compose([transforms.ToTensor(),
                                     transforms.Normalize((0.5, 0.5, 0.5),
                                                          (0.5, 0.5, 0.5))])

train_set = datasets.CIFAR10('~/.pytorch/cifar_data/', download=True, train=True, \
                           transform=transform) 
train_loader = torch.utils.data.DataLoader(train_set, batch_size=8, shuffle=True)

test_set = datasets.CIFAR10('~/.pytorch/cifar_data/', download=True, train=False, \
                          transform=test_transform)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=32, shuffle=True)

Files already downloaded and verified
Files already downloaded and verified


### Define the function we will need

- Softmax Function: This function will turn our output layer into probabilities

<br>
$$ Softmax(x_j)=\frac{e^{x_j}}{\sum_{k}^{K}e^{x_k}}$$
<br>


### Define our model
- Here we define the layers of our model as well as how the inputs should flow through

- We will use Pytorch to make this easy

In [0]:
import torch
from torch import nn
import torch.nn.functional as F

class Network(nn.Module):
    
    def __init__(self):
        
        super(Network, self).__init__()
        
        # Convolutional layers
        # This is where the real work will need to be done
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=16, kernel_size=3, padding=1)
        
        # Our pooling function; Try other types of pooling functions!
        self.pool = nn.MaxPool2d(2, 2)
        
        # MLP Layers
        
        # Remember, the input to our MLP is 
        # (number_of_channels * (feaure_map_height * feature_map_width))
        self.fc1 = nn.Linear(16 * (16 * 16), 512)
        self.fc2 = nn.Linear(512, 512)
        self.fc3 = nn.Linear(512, 10)
        
        self.dropout = nn.Dropout(0.4)
    
    def forward(self, x):
        
        x = torch.relu(self.conv1(x))
        x = torch.relu(self.conv2(x))
        x = self.pool(x)
        
        # Remember, the input to our MLP is 
        # (number_of_channels * (feaure_map_height * feature_map_width))
        x = x.view(-1, 16 * (16 * 16))
        
        # Feed forward network
        x = self.dropout(F.relu(self.fc1(x)))
        x = self.dropout(F.relu(self.fc2(x)))
        x = F.log_softmax(self.fc3(x), dim=1)
        
        return x

### What is dropout?

- This forces the neural network to have all neurons to be important in its decision making

- Without it, the network could decide that only a handful of neurons will decide what the output is

<br>

<img src='assets/dropoutAnimation.gif' style="width: 75%; height: 75%">

In [0]:
## Here we determine if we have a gpu to use
use_cuda = False
if torch.cuda.is_available:
    use_cuda = True

## Define the Neural Network and Hyperparameters

In [5]:
# Instantiate our network
net = Network()

# Determine if we can move our model to a gpu
if use_cuda:
    print('GPU detected. Moving model to CUDA')
    net = net.cuda()

# Hyperparameters
epochs = 10
learning_rate = 0.01 # IMPORTANT: This is way too high; try experimenting

# We use this as our loss function because we used log_softmax as our activation function
# in the output layer of our network. When we use this loss function combined with
# log_softmax it is exactly the same as if we used softmax and cross_entropy
criterion = nn.NLLLoss()

# Try experimenting with weight_decay too!
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)

GPU detected. Moving model to CUDA


RuntimeError: ignored

### Time to Train!
If you change absolutely nothing, you will not get a good model (probably no better than random guesses).
Try experimenting with hyperparmeters and model architecture to get above 70%!

In [0]:
for e in range(1, epochs+1):
    
    # Turn dropout on
    net.train()
    
    running_loss = 0
    
    # Train our model in batches of 64 images, labels
    for images, targets in train_loader:
        
        optimizer.zero_grad()
        
        # If we are using a gpu, we need to move the images and labels to it
        if use_cuda:
            images, targets = images.cuda(), targets.cuda()
        
        # Forward pass
        output = net.forward(images)
        
        # Calculate loss
        loss = criterion(output, targets)
        
        # Backpropagate
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
    # Turn off dropout in our model
    net.eval()
    
    accuracy = 0
    
    for images, labels in test_loader:
        
        if use_cuda:
            images, labels = images.cuda(), labels.cuda()
            
        log_probabilities = net.forward(images)
        probabilities = torch.exp(log_probabilities)
        _, top_class = probabilities.topk(1, dim=1)
        equality = top_class == labels.view(*top_class.shape)
        accuracy += torch.mean(equality.type(torch.FloatTensor))
        
    accuracy = accuracy / len(test_loader)
    print('Epoch: {} \tTraining Loss: {:3f}\tTesting Accuracy: {:.3f}%'.format(e, running_loss, (accuracy * 100.0)))
    print()


In [0]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import helper

net = net.cpu()
net.eval()

dataiter = iter(test_loader)
images, labels = dataiter.next()
img = images[0].unsqueeze(0)

# Calculate the class probabilities (softmax) for img
ps = torch.exp(net.forward(img))

# Plot the image and probabilities
helper.view_classify(img[0], ps)